<a href="https://colab.research.google.com/github/Pakhi27/Object-Detction-using-Yolo-v5--Raccon-Images-Dataset-/blob/main/Object_Detction_using_Yolo_v5_(Raccon_Images_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

YOLO Architecture (You Only Look Once)
YOLO is an efficient and accurate object detection model that detects multiple objects in images in a single forward pass. Below is an overview of the architecture and how it works.

1. YOLO Architecture
The YOLO architecture consists of three key parts: feature extraction, bounding box prediction, and class prediction.

a) Backbone (Feature Extraction)
Convolutional Layers: YOLO uses convolutional layers to extract features from the input image. It applies multiple convolutional layers with different filter sizes to capture spatial features at different levels of granularity.
Grid Division: The input image is divided into an
𝑆
×
𝑆
S×S grid. Each grid cell is responsible for detecting objects whose center falls within that cell.
b) Head (Bounding Box Prediction)
For each grid cell, the model predicts several bounding boxes, each with:
Coordinates:
(
𝑥
,
𝑦
,
𝑤
,
ℎ
)
(x,y,w,h), where
𝑥
x and
𝑦
y represent the center of the bounding box relative to the grid cell, and
𝑤
w and
ℎ
h are the width and height of the box relative to the image.
Confidence Score: A score that reflects how confident the model is that the bounding box contains an object.
c) Output Layer (Class Prediction)
For each bounding box, YOLO also predicts the class probabilities of the object (e.g., person, dog, car) using a softmax function. This is based on the features extracted from the image.
The output is a
𝑆
×
𝑆
×
(
𝐵
×
5
+
𝐶
)
S×S×(B×5+C) tensor, where:
𝑆
×
𝑆
S×S is the grid size.
𝐵
B is the number of bounding boxes per grid cell.
5
5 corresponds to 4 bounding box attributes (x, y, w, h) and the confidence score.
𝐶
C is the number of object classes.


YOLOv3 Architecture Example (A Common YOLO Version)
Input Layer: The input is an image, typically resized to 416x416 or 608x608 pixels.
Darknet-53 Backbone: YOLOv3 uses the Darknet-53 architecture, which consists of:
53 convolutional layers for feature extraction.
Residual connections (inspired by ResNet) to allow better gradient flow during training.
Feature Pyramid Network (FPN): This is used for multi-scale detection to improve the detection of small objects.
It detects objects at 3 different scales from the feature maps.
Bounding Box Prediction: For each scale, the network predicts bounding boxes with objectness scores and class probabilities.
Non-Maximum Suppression (NMS): YOLO applies NMS to remove duplicate detections and keep only the best bounding box for each object.


2. How YOLO Works
Step 1: Image Division
YOLO divides the input image into an
𝑆
×
𝑆
S×S grid. Each grid cell is responsible for detecting objects whose center lies within the cell.
Step 2: Bounding Box Prediction
Each grid cell predicts several bounding boxes. For each bounding box, YOLO predicts:
The coordinates of the box relative to the grid cell.
The width and height relative to the entire image.
A confidence score representing how confident the model is that the box contains an object and how accurate it thinks the prediction is.
Step 3: Class Prediction
For each bounding box, the model predicts a class probability distribution using softmax activation. The output is the most likely class for the detected object.
Step 4: Final Output
The final output is a set of bounding boxes with class labels and confidence scores. YOLO then filters out low-confidence boxes using a confidence threshold.
Step 5: Non-Maximum Suppression (NMS)
To avoid multiple boxes around the same object, YOLO applies non-maximum suppression. It keeps the bounding box with the highest confidence score and removes overlapping boxes based on a threshold for Intersection over Union (IoU).

Racoon Images Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/ultralytics/yolov5  # Clone the YOLOv5 repo
%cd yolov5
!pip install -r requirements.txt  # Install the necessary libraries


Cloning into 'yolov5'...
remote: Enumerating objects: 16954, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 16954 (delta 73), reused 94 (delta 45), pack-reused 16805 (from 1)
Receiving objects: 100% (16954/16954), 15.71 MiB | 15.91 MiB/s, done.
Resolving deltas: 100% (11606/11606), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [4]:

!pip install torch torchvision opencv-python


In [1]:
!git clone https://github.com/ultralytics/yolov5
!cd yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
import os
import pandas as pd

# Load the CSV file
csv_file = '/content/drive/MyDrive/Racoon Images/train_labels_.csv'  # Replace with actual path
images_folder = '/content/drive/MyDrive/Racoon Images/images'  # Replace with actual path
output_label_folder = 'labels'  # Folder to store YOLO format labels
os.makedirs(output_label_folder, exist_ok=True)

# Read the CSV file
df = pd.read_csv(csv_file)

# Convert each row to YOLO format
for index, row in df.iterrows():
    # Image dimensions
    img_width = row['width']
    img_height = row['height']

    # Bounding box coordinates
    xmin = row['xmin']
    ymin = row['ymin']
    xmax = row['xmax']
    ymax = row['ymax']

    # Convert to YOLO format
    x_center = ((xmin + xmax) / 2) / img_width
    y_center = ((ymin + ymax) / 2) / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height

    # Get the class (assuming 'raccoon' is the only class, class index is 0)
    class_id = row['class']

    # Create label file in YOLO format
    image_filename = row['filename']
    label_filename = os.path.splitext(image_filename)[0] + ".txt"
    label_filepath = os.path.join(output_label_folder, label_filename)

    # Write to the label file
    with open(label_filepath, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print("Conversion to YOLO format complete.")


Conversion to YOLO format complete.


In [7]:

yaml_content = """
train: '/content/drive/MyDrive/Racoon Images/images'  # Path to the folder with training images
val: '/content/drive/MyDrive/Racoon Images/images'    # Using same folder for validation

nc: 1
names: ['raccoon']
"""

# Write the YAML content to a file
with open('raccoon.yaml', 'w') as file:
    file.write(yaml_content)

print("raccoon.yaml file created.")



raccoon.yaml file created.


In [8]:
!python /content/yolov5/train.py --img 640 --batch 16 --epochs 50 --data raccoon.yaml --weights yolov5s.pt


2024-09-13 18:33:29.010588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 18:33:29.046219: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 18:33:29.056763: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=raccoon.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single

In [9]:

import torch
import os
from PIL import Image

# Load the pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 small pre-trained model

# Path to your raccoon images folder
images_folder = '/content/drive/MyDrive/Racoon Images/images'  # Replace with the actual path
output_folder = 'output_images_folder'  # Folder to save output images with detections

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all images in the folder
for image_name in os.listdir(images_folder):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
        image_path = os.path.join(images_folder, image_name)

        # Load the image
        img = Image.open(image_path)

        # Perform object detection
        results = model(img)

        # Print detection results
        results.print()

        # Save output image with bounding boxes
        output_image_path = os.path.join(output_folder, image_name)
        results.save(save_dir=output_folder)

        print(f"Processed {image_name} and saved to {output_image_path}")

print("Object detection complete for all images.")


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-9-13 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWa

Processed raccoon-104.jpg and saved to output_images_folder/raccoon-104.jpg


image 1/1: 640x426 1 bear
Speed: 19.5ms pre-process, 383.2ms inference, 3.7ms NMS per image at shape (1, 3, 640, 448)
Saved 1 image to output_images_folder3
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed raccoon-115.jpg and saved to output_images_folder/raccoon-115.jpg


image 1/1: 960x720 1 person
Speed: 40.0ms pre-process, 385.9ms inference, 4.5ms NMS per image at shape (1, 3, 640, 480)
Saved 1 image to output_images_folder4
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Processed raccoon-79.jpg and saved to output_images_folder/raccoon-79.jpg


image 1/1: 360x480 1 bowl
Speed: 7.3ms pre-process, 372.2ms inference, 1.7ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to output_images_folder200
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed raccoon-98.jpg and saved to output_images_folder/raccoon-98.jpg


image 1/1: 450x600 1 bear
Speed: 8.7ms pre-process, 383.7ms inference, 1.8ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to output_images_folder201


Processed raccoon-81.jpg and saved to output_images_folder/raccoon-81.jpg
Object detection complete for all images.


In [12]:
import pandas as pd
import os

def convert_csv_to_yolo(csv_file, images_dir, output_dir):
    df = pd.read_csv(csv_file)
    class_mapping = {cls: idx for idx, cls in enumerate(df['class'].unique())}

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in df.iterrows():
        image_file = row['filename']
        class_id = class_mapping[row['class']]
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']

        # Convert to YOLO format
        width = xmax - xmin
        height = ymax - ymin
        x_center = xmin + width / 2
        y_center = ymin + height / 2

        # Normalize values
        image_path = os.path.join(images_dir, image_file)
        img = Image.open(image_path)
        img_width, img_height = img.size
        x_center /= img_width
        y_center /= img_height
        width /= img_width
        height /= img_height

        # Write YOLO format annotation
        txt_file = os.path.join(output_dir, os.path.splitext(image_file)[0] + '.txt')
        with open(txt_file, 'a') as f:
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

convert_csv_to_yolo('/content/drive/MyDrive/Racoon Images/train_labels_.csv', '/content/drive/MyDrive/Racoon Images/images/', 'labels_yolo/')



In [13]:
import torch
import os
from PIL import Image

# Load the pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5 small pre-trained model

# Path to your raccoon images folder
images_folder = '/content/drive/MyDrive/Racoon Images/images'  # Replace with the actual path
output_folder = '/content/output_images_folder'  # Folder to save output images with detections

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all images in the folder
for image_name in os.listdir(images_folder):
    if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Check for image files
        image_path = os.path.join(images_folder, image_name)

        # Load the image
        img = Image.open(image_path)

        # Perform object detection
        results = model(img)

        # Print detection results
        results.print()

        # Save output image with bounding boxes
        output_image_path = os.path.join(output_folder, image_name)
        results.save(save_dir=output_folder)

        print(f"Processed {image_name} and saved to {output_image_path}")

print("Object detection complete for all images.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-13 Python-3.10.12 torch-2.4.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 304x600 1 person, 1 dog, 1 potted plant, 1 bed
Speed: 7.6ms pre-process, 250.2ms inference, 1.5ms NMS per image at shape (1, 3, 352, 640)
Saved 1 image to /content/output_images_folder202
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed raccoon-104.jpg and saved to /content/output_images_folder/raccoon-104.jpg


image 1/1: 640x426 1 bear
Speed: 7.8ms pre-process, 333.8ms inference, 1.5ms NMS per image at shape (1, 3, 640, 448)
Saved 1 image to /content/output_images_folder203
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed raccoon-115.jpg and saved to /content/output_images_folder/raccoon-115.jpg


image 1/1: 960x720 1 person
Speed: 68.8ms pre-process, 555.8ms inference, 2.1ms NMS per image at shape (1, 3, 640, 480)
Saved 1 image to /content/output_images_folder204
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

Processed raccoon-79.jpg and saved to /content/output_images_folder/raccoon-79.jpg


image 1/1: 360x480 1 bowl
Speed: 10.7ms pre-process, 591.4ms inference, 6.5ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to /content/output_images_folder400
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Processed raccoon-98.jpg and saved to /content/output_images_folder/raccoon-98.jpg


image 1/1: 450x600 1 bear
Speed: 11.5ms pre-process, 565.1ms inference, 5.0ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to /content/output_images_folder401


Processed raccoon-81.jpg and saved to /content/output_images_folder/raccoon-81.jpg
Object detection complete for all images.
